In [180]:
import sys
!{sys.executable} -m pip install -q langchain_core langchain-openai requests

In [181]:
from typing_extensions import Annotated
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, InjectedToolArg
from langchain_core.messages import HumanMessage, ToolMessage
from typing import Annotated
import requests

# Tool Creation

In [182]:

# tool create

# tool1 : To fetch currency conversion factor
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function takes base_currency and target_currency and fetches the currency_factor  """
    url = f"https://v6.exchangerate-api.com/v6/85398964108a97d9af3c818a/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    data = response.json()
    print(data)
    return data["conversion_rate"]

# tool2 : To multiply currency conversion factor with base amount
@tool
def currency_conversion(base_currency_value: float, conversion_rate:Annotated[float,InjectedToolArg]) -> float:
    """ With the given base_currency with the conversion rate to give the amount in target_currency """
    return base_currency_value * conversion_rate


In [183]:
# currency_conversion_rate= get_conversion_factor.invoke({'base_currency':'USD', 'target_currency' :'INR'})
# print(currency_conversion_rate)

In [184]:
# print(currency_conversion.invoke({'base_currency_value':100, 'conversion_rate':currency_conversion_rate}))

# Tool Binding

In [185]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0, api_key="")

In [186]:
llm_with_tools = llm.bind_tools([currency_conversion,get_conversion_factor])

In [187]:
llm_with_tools

RunnableBinding(bound=ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x7916bb833530>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7916bb833d70>, root_client=<openai.OpenAI object at 0x7916bb8330b0>, root_async_client=<openai.AsyncOpenAI object at 0x7916bb832420>, model_name='gpt-3.5-turbo-16k', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), kwargs={'tools': [{'type': 'function', 'function': {'name': 'currency_conversion', 'description': 'With the given base_currency with the conversion rate to give the amount in target_currency', 'parameters': {'properties': {'base_currency_value': {'type': 'number'}}, 'required': ['base_currency_value'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function takes base_currency and target_currency and fetches the currency_factor', 'parameters': {'p

# Tool Calling

In [188]:
message_list = []
message1 = HumanMessage(' What is the currency exchange rate between USD and CAD and based on that can you convert 1500 USD to CAD')
message_list.append(message1)
message_list

[HumanMessage(content=' What is the currency exchange rate between USD and CAD and based on that can you convert 1500 USD to CAD', additional_kwargs={}, response_metadata={})]

In [189]:
ai_message = llm_with_tools.invoke(message_list)

In [190]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'CAD'},
  'id': 'call_uUSGap8uBHCPTGPCskuvisLX',
  'type': 'tool_call'}]

# Tool Execution

In [191]:
import json
for tool_call in ai_message.tool_calls:
  # execute first tool to get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # print(json.loads(tool_message1.content))
        # fetch this conversion rate
        conversion_rate = json.loads(tool_message1.content)
        print(conversion_rate)
        # append this tool message to message_list
        message_list.append(tool_message1)
  # execute 2nd tool using conversion rate
    if tool_call['name'] == 'currency_conversion':
      print("Data tere naam")
       # fetch the current argument
      tool_call['args']['conversion_rate'] = conversion_rate
      tool_message2 = currency_conversion.invoke(tool_call)
       # append this tool message to message_list
      message_list.append(tool_message2)

{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1764288002, 'time_last_update_utc': 'Fri, 28 Nov 2025 00:00:02 +0000', 'time_next_update_unix': 1764374402, 'time_next_update_utc': 'Sat, 29 Nov 2025 00:00:02 +0000', 'base_code': 'USD', 'target_code': 'CAD', 'conversion_rate': 1.4032}
1.4032


In [192]:
message_list

[HumanMessage(content=' What is the currency exchange rate between USD and CAD and based on that can you convert 1500 USD to CAD', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='1.4032', name='get_conversion_factor', tool_call_id='call_uUSGap8uBHCPTGPCskuvisLX')]

In [193]:
llm_with_tools.invoke (message_list).content

BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[1].role', 'code': None}}